In [ ]:
import sys
import os
import random
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn import svm
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
import scipy

from PIL import Image

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def read_image(dirpath):
    image_data = []
    
    for file in os.listdir(dirpath):
        if os.path.isfile(os.path.join(dirpath, file)):
            c = os.path.basename(file)
            filename = dirpath + '/' + c
            image = Image.open(filename).resize((32, 32)).convert('L')
            image_arr = np.array(image).reshape(1, 1024)
            image_data.append(image_arr)
    
    return image_data
            

In [ ]:
def process_image(folderpath):
    data_all = []
    for i in range(1, 26):
        ### '/home/x/PR_CA/PIE/'
        dirpath = folderpath + str(i)  
        image_list = read_image(dirpath)
        data_image = np.concatenate(image_list, axis=0)
        data_all.append(data_image)
    return data_all
    

In [ ]:
def train_test_split(data, label):
    proportion = 0.7
    lens = data.shape[0]
    print(lens)
    idx = random.sample(range(0, lens), int(proportion*lens))
    idx = sorted(idx)
    
    train_x, train_y, test_x, test_y = [], [], [], []
    for i in range(lens):
        if i in idx:
            train_x.append(data[i].reshape(1, -1))
            train_y.append(label[i].reshape(1, -1))
        else:
            test_x.append(data[i].reshape(1, -1))
            test_y.append(label[i].reshape(1, -1))
    
#     train_x = [data[i].reshape(1, -1) for i in idx]
#     train_y = [label[i].reshape(1, -1) for i in idx]
#     test_x = [data[i].reshape(1, -1) for i in range(lens) if i not in idx]
#     test_y = [label[i].reshape(1, -1) for i in range(lens) if i not in idx]
    
    trainx = np.concatenate(train_x)
    trainy = np.concatenate(train_y)
    testx = np.concatenate(test_x)
    testy = np.concatenate(test_y)
    return trainx, trainy, testx, testy

In [ ]:
def PCA_(dataset, n_component):
    N, Fea = dataset.shape
    x_mean = np.mean(dataset, 0)
    x_norm = dataset - x_mean
    
    co_var = np.zeros([Fea, Fea])
    co_var = np.dot(x_norm.T, x_norm)
    
    eig_val, eig_vec = np.linalg.eig(co_var)
    eig_pair = [[np.abs(eig_val[i]), eig_vec[:, i]] for i in range(Fea)]
    
    feature = np.array([pair[1] for pair in eig_pair[:n_component]])
    data = np.dot(x_norm, feature.T)
    
    return np.real(data)

In [ ]:
def PCA_Split(image_all, label_image, self_image, label_self, n_component):
    image_all = PCA_(image_all, n_component)
    self_image = PCA_(self_image, n_component)
    trainx, trainy, testx, testy = train_test_split(image_all, label_image)
    trainx_self, trainy_self, testx_self, testy_self = train_test_split(self_image, label_self)

    # Training data and testing data
    train_x = np.concatenate([trainx, trainx_self])
    train_y = np.concatenate([trainy, trainy_self])
    test_x = np.concatenate([testx, testx_self])
    test_y = np.concatenate([testy, testy_self])
    data_pro = [train_x, train_y, test_x, test_y]
    return data_pro

In [ ]:
train_x = np.loadtxt('/content/drive/MyDrive/Colab Notebooks/train_x')
train_y = np.loadtxt('/content/drive/MyDrive/Colab Notebooks/train_y').astype(int)
test_x = np.loadtxt('/content/drive/MyDrive/Colab Notebooks/test_x')
test_y = np.loadtxt('/content/drive/MyDrive/Colab Notebooks/test_y').astype(int)


In [ ]:
train_x[0]

array([36., 35., 35., ..., 11., 12.,  9.])

In [ ]:
import torch
import torch.nn as nn
import torchvision
import torch.nn.functional as F
import torch.utils.data as Data
import torch.optim as optim

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
print(torch.__version__)


cuda
1.12.1+cu113


In [ ]:
### CNN
class CNN(nn.Module):
  def __init__(self, num_classes):
    super(CNN, self).__init__()
    self.conv1 = nn.Sequential(
        nn.Conv2d(
            in_channels=1,
            out_channels=20,
            kernel_size=5,
            stride=1,
            padding=2,
        ),
        nn.ReLU(),
        nn.BatchNorm2d(20),
        nn.MaxPool2d(kernel_size=2, stride=2),
    )

    self.conv2 = nn.Sequential(
        nn.Conv2d(
            in_channels=20,
            out_channels=50,
            kernel_size=5,
            stride=1,
            padding=2,
        ),
        nn.ReLU(),
        nn.BatchNorm2d(50),
        nn.MaxPool2d(kernel_size=2, stride=2),
    )

    self.out1 = nn.Linear(50*8*8, 500)
    self.out2 = nn.Linear(500, num_classes)

  def forward(self, x):
    x = self.conv1(x)
    x = self.conv2(x)
    x = x.reshape(x.shape[0], -1)
    x = F.relu(self.out1(x))
    x = self.out2(x)

    return x

model = CNN(num_classes=26).to(device)
print(model)


CNN(
  (conv1): Sequential(
    (0): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): BatchNorm2d(20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv2d(20, 50, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): BatchNorm2d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (out1): Linear(in_features=3200, out_features=500, bias=True)
  (out2): Linear(in_features=500, out_features=26, bias=True)
)


In [ ]:
### Hypeparameters
num_samples = train_x.shape[0]
num_classes = 26
batch_size = 128
epochs = 200
learning_rate = 1e-3

optimizer = optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()


In [ ]:
### Dataloader

train_data = torch.tensor(train_x.reshape(num_samples, 1, 32, 32)).float().to(device)
train_label = np.zeros([num_samples, num_classes])
for y in range(train_y.shape[0]):
  train_label[y][train_y[y]] = 1

train_label = torch.tensor(train_label).float().to(device)

training = Data.TensorDataset(train_data, train_label)
train_loader = Data.DataLoader(training, batch_size=batch_size, shuffle=True)


In [ ]:
train_label.shape

torch.Size([2982, 26])

In [ ]:
test_data = torch.tensor(test_x.reshape(test_x.shape[0], 1, 32, 32)).float().to(device)
test_label = torch.tensor(test_y).float().to(device)
testing = Data.TensorDataset(test_data, test_label)
test_loader = Data.DataLoader(dataset=testing, batch_size=batch_size, shuffle=False)

def check_accuracy(loader, model):
    model.eval()
    # print("checking accracy on testing data")

    num_correct = 0
    num_samples = 0
    res_class = []

    with torch.no_grad():
      for x, y in loader:
        x = x.to(device)
        y = y.to(device)

        score = model(x)
        _, prediction = score.max(1)

        prediction = prediction.to(device='cpu').numpy()
        res_class.append(prediction)

        label = y.to(device='cpu').numpy().reshape(1, prediction.shape[0])
        num_correct += np.sum((prediction==label[0]))
        num_samples += prediction.shape[0]
    
    res = np.concatenate(res_class)
    acc = num_correct/num_samples
    model.train()
    return res, acc



In [ ]:
### Training
model.train()

for i in range(epochs):
  for barch_idx, (data, target) in enumerate(train_loader):
    data = data.to(device)
    target = target.to(device)

    score = model(data)
    loss = criterion(score, target)
    loss.requires_grad_(True)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  _, acc = check_accuracy(test_loader, model)
  print("epoch:", i, "loss:", loss.item(), "accuracy:", acc)




epoch: 0 loss: 0.17845569550991058 accuracy: 0.9460093896713615
epoch: 1 loss: 0.12098465859889984 accuracy: 0.9804381846635368
epoch: 2 loss: 0.013142694719135761 accuracy: 0.9859154929577465
epoch: 3 loss: 0.003539911936968565 accuracy: 0.9929577464788732
epoch: 4 loss: 0.001076349290087819 accuracy: 0.9929577464788732
epoch: 5 loss: 0.0008281811606138945 accuracy: 0.9937402190923318
epoch: 6 loss: 0.12909334897994995 accuracy: 0.986697965571205
epoch: 7 loss: 0.049006540328264236 accuracy: 0.9843505477308294
epoch: 8 loss: 0.012225382961332798 accuracy: 0.9874804381846636
epoch: 9 loss: 0.0008614477119408548 accuracy: 0.986697965571205
epoch: 10 loss: 0.001979112159460783 accuracy: 0.9937402190923318
epoch: 11 loss: 0.0008049123571254313 accuracy: 0.9906103286384976
epoch: 12 loss: 0.000340265833074227 accuracy: 0.9945226917057903
epoch: 13 loss: 0.00019660453835967928 accuracy: 0.9945226917057903
epoch: 14 loss: 9.137872257269919e-05 accuracy: 0.9953051643192489
epoch: 15 loss: 6.0

In [ ]:
test_data = torch.tensor(test_x.reshape(test_x.shape[0], 1, 32, 32)).float().to(device)
test_label = torch.tensor(test_y).float().to(device)
testing = Data.TensorDataset(test_data, test_label)
test_loader = Data.DataLoader(dataset=testing, batch_size=batch_size, shuffle=False)

def check_accuracy(loader, model):
    model.eval()
    print("checking accracy on testing data")

    num_correct = 0
    num_samples = 0
    res_class = []

    with torch.no_grad():
      for x, y in loader:
        x = x.to(device)
        y = y.to(device)

        score = model(x)
        _, prediction = score.max(1)

        prediction = prediction.to(device='cpu').numpy()
        res_class.append(prediction)

        label = y.to(device='cpu').numpy().reshape(1, prediction.shape[0])
        num_correct += np.sum((prediction==label[0]))
        num_samples += prediction.shape[0]
    
    res = np.concatenate(res_class)
    acc = num_correct/num_samples
    return res, acc

res, acc = check_accuracy(test_loader, model)
print("The result of classification is: ", res)
print("The accuracy of results is: ", acc)
torch.save(model.state_dict(), '/content/drive/MyDrive/Colab Notebooks/pr_cnn.pth')

checking accracy on testing data
The result of classification is:  [ 0  0  0 ... 25 25 25]
The accuracy of results is:  0.9945226917057903
